###  **Essential Functionality**

In [2]:
import numpy as np
import pandas as pd

### Reindexing :
- which means to create a new object with the values rearranged to align with the new index.


In [8]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])
obj2 = obj.reindex(["d", "b", "a", "c","d","e"])
obj2

d    4.5
b    7.2
a   -5.3
c    3.6
d    4.5
e    NaN
dtype: float64

In [14]:
obj3 = pd.Series(["blue", "purple", "yellow"], index=[0, 2, 4])
obj3.reindex(np.arange(6),method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

The `ffill` *(forward fill)* method fills in missing values (i.e., for indices 1, 3, and 5) by propagating the last valid value forward.

In [15]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=["a", "c", "d"],
                     columns=["Ohio", "Texas", "California"])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


### **reindexing functions :**

| Argument     | Description                                                                                                                                          |
|--------------|------------------------------------------------------------------------------------------------------------------------------------------------------|
| `labels`     | New sequence to use as an index. Can be an `Index` instance or any other sequence-like Python data structure. An `Index` will be used exactly as is without any copying. |
| `index`      | Use the passed sequence as the new index labels.                                                                                                     |
| `columns`    | Use the passed sequence as the new column labels.                                                                                                    |
| `axis`       | The axis to reindex, whether "index" (rows) or "columns". The default is "index". You can alternately do `reindex(index=new_labels)` or `reindex(columns=new_labels)`. |
| `method`     | Interpolation (fill) method; `"ffill"` fills forward, while `"bfill"` fills backward.                                                                |
| `fill_value` | Substitute value to use when introducing missing data by reindexing. Use `fill_value="missing"` (the default behavior) when you want absent labels to have null values. |
| `limit`      | When forward filling or backfilling, the maximum size gap (in number of elements) to fill.                                                           |
| `tolerance`  | When forward filling or backfilling, the maximum size gap (in absolute numeric distance) to fill for inexact matches.                                |
| `level`      | Match simple `Index` on the level of `MultiIndex`; otherwise select subset of.                                                                       |
| `copy`       | If `True`, always copy underlying data even if the new index is equivalent to the old index; if `False`, do not copy the data when the indexes are equivalent. |
